In [ ]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip, DeltaTable

from samplepackage.tables import People, Scores
from samplepackage.utils import null_safe_join, drop_managed_table

In [ ]:
builder = SparkSession.builder.appName("MyApp") \
    .config(
        "spark.sql.extensions",
        "io.delta.sql.DeltaSparkSessionExtension") \
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config(
        "spark.sql.warehouse.dir",
        "/workspaces/spark-sandbox/sample_entrypoints/spark-warehouse/")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [ ]:
people_df = People(spark).get_df()
scores_df = Scores(spark).get_df()

join_conditions = [("id", "person_id"),]

In [ ]:
full_df = null_safe_join(people_df, scores_df, join_conditions)

In [ ]:
full_df.show()

In [ ]:
table_name = "full_scores"

full_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

In [ ]:
spark.sql(f"SELECT * FROM {table_name}").show()

In [ ]:
# Clean-up
drop_managed_table(spark, table_name)